**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [29]:
import os
import binascii
# import csv
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
import sys
from Crypto.Hash import keccak
import integration_helpers
from name_generator import left, right
import json
from utilities import *
from dump_to_json import *
from iroha_helper import *

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]
iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))

In [30]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

In [31]:
# - Deploy contract
hash = create_account_contract()

2025-01-14 21:27:21.086 | INFO     | integration_helpers:tracer:34 - 	Entering "create_account_contract"
2025-01-14 21:27:21.106 | INFO     | iroha_helper:create_account_contract:41 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 21:27:21.110 | INFO     | iroha_helper:create_account_contract:41 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:27:22.235 | INFO     | iroha_helper:create_account_contract:41 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:27:22.265 | INFO     | iroha_helper:create_account_contract:41 - ('COMMITTED', 5, 0)
2025-01-14 21:27:22.267 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_account_contract"


2 - Creates a user account into the blockchain.

In [32]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

@integration_helpers.trace
def get_account(address):
    params = integration_helpers.get_first_four_bytes_of_keccak(b"getAccount(string)")
    no_of_param = 1
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(f"{user_account_short_id}@{DOMAIN}")  # account id
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        logger.info(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_user_account(address, user_account_short_id, DOMAIN, user_public_key, user_account)

# - Get account info
hash = get_account(address)
address = integration_helpers.get_engine_receipts_result(hash)


2025-01-14 21:27:22.326 | INFO     | __main__:<module>:37 - User Role: publisher
2025-01-14 21:27:22.328 | INFO     | __main__:<module>:48 - Creating account with name: jolly_carson
2025-01-14 21:27:22.330 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 21:27:22.350 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-14 21:27:22.351 | INFO     | integration_helpers:tracer:34 - 	Entering "create_user_account"
2025-01-14 21:27:22.367 | INFO     | iroha_helper:create_user_account:81 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 21:27:22.369 | INFO     | iroha_helper:create_user_account:81 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:27:25.286 | INFO     | iroha_helper:create_user_account:81 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:27:25.311 | INFO     | iroha_helper:create_user_account:81 - ('COMMITTED', 5, 0)
2025-01-14 21:27:25.314 | INFO     | dump_to_json:dump_to_json_ld

2025-01-11 20:09:13.535 | INFO     | iroha_helper:create_user_account:84 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 20:09:13.537 | INFO     | iroha_helper:create_user_account:84 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 20:09:16.427 | INFO     | iroha_helper:create_user_account:84 - ('STATEFUL_VALIDATION_FAILED', 2, 3)
2025-01-11 20:09:16.466 | INFO     | iroha_helper:create_user_account:84 - ('REJECTED', 4, 0)
2025-01-11 20:09:16.472 | INFO     | dump_to_json:dump_to_json_ld:77 - Appended new entry to file 'datasets/accounts.json', current total entries: 22
2025-01-11 20:09:16.474 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_user_account"
2025-01-11 20:09:16.475 | INFO     | integration_helpers:tracer:34 - 	Entering "get_account"
2025-01-11 20:09:16.493 | INFO     | __main__:get_account:23 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 20:09:16.495 | INFO     | __main__:get_account:23 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 20:09:19.4

3 - Queries the blockchain to confirm the succesful creation of the account.

In [33]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{user_account_short_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
logger.info(f'Account id = {user_account_short_id}@{DOMAIN}, details = {data.detail}')

2025-01-14 21:27:28.437 | INFO     | __main__:<module>:7 - Account id = jolly_carson@test, details = {}


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting.

In [34]:
# - Deploys the contract
hash = create_detail_contract()
address = integration_helpers.get_engine_receipts_address(hash)
integration_helpers.get_engine_receipts_result(hash)

2025-01-14 21:27:28.446 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-14 21:27:28.467 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 21:27:28.470 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:27:31.421 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 21:27:31.458 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-14 21:27:31.459 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-14 21:27:31.461 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 21:27:31.480 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-14 21:27:31.483 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-14 21:27:31.5

2025-01-11 20:09:22.574 | INFO     | iroha_helper:create_detail_contract:58 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 20:09:22.603 | INFO     | iroha_helper:create_detail_contract:58 - ('COMMITTED', 5, 0)
2025-01-11 20:09:22.605 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-11 20:09:22.607 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 20:09:22.626 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 20:09:22.627 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-11 20:09:22.645 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-11 20:09:22.646 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key

In [35]:
process_account(address, f"{user_account_short_id}@{DOMAIN}")


2025-01-14 21:27:31.512 | INFO     | iroha_helper:process_account:318 - {'@type': 'foaf:Person', 'foaf:name': 'Jolly Carson', 'foaf:mbox': 'jolly_carson@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'School of Economic Sciences'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '8740-6568-5698-X'}, 'foaf:holdsAccount': {'schema:identifier': 'jolly_carson@test', 'schema:roleName': 'publisher', 'schema:publicKey': '17ee3952531c1aa128809122466bd76c70e4179877c32da4243c7776d2f9dd88'}}
2025-01-14 21:27:31.544 | INFO     | iroha_helper:process_account:322 - user_json_ld_cid :, QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU
2025-01-14 21:27:31.545 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-14 21:27:31.561 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-14 21:27:31.563 | INFO     | iroha_helper:set_account_detail:298 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 21:27:34.484 | 

6 - Queries the blockchain again to confirm the succesfull details setting.

In [36]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{user_account_short_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
details = data.detail
logger.info(details)
logger.info(f'User Account id = {user_account_short_id}@{DOMAIN}, {details}')

2025-01-14 21:27:34.549 | INFO     | __main__:<module>:8 - { "admin@test" : { "user_json_ld_cid" : "QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU" } }
2025-01-14 21:27:34.550 | INFO     | __main__:<module>:9 - User Account id = jolly_carson@test, { "admin@test" : { "user_json_ld_cid" : "QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU" } }


7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [37]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
user_metadata_cid = details_dict["admin@test"]["user_json_ld_cid"]
logger.info(f"User Metadata CID: {user_metadata_cid}")

user_metadata = download_json_from_ipfs(user_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-14 21:27:34.561 | INFO     | __main__:<module>:6 - User Metadata CID: QmaooJHb4vCziESs7MqcC4LhnR7RfMMhG3iQvcpUyxrUHU
2025-01-14 21:27:34.570 | INFO     | __main__:<module>:9 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'jolly_carson@test', 'schema:publicKey': '17ee3952531c1aa128809122466bd76c70e4179877c32da4243c7776d2f9dd88', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'jolly_carson@email.com', 'foaf:name': 'Jolly Carson', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'School of Economic Sciences'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '8740-6568-5698-X'}}
